# some AXI designs

The CoHDL standard library module `std.axi` provides helper classes for AXI4 lite interfaces. While this is experimental and mostly untested, the following examples still show, what a real world design using CoHDL for AXI transactions could look like.

In [1]:
# imports needed by the following cells
from cohdl import Entity, Port, Signal, Variable
from cohdl import Bit, BitVector, Unsigned, Array, Null

from cohdl import std
from cohdl.std.axi.axi4_light import Axi4Light
from cohdl.std.axi.axi4_light.interconnect import Interconnect

This section defines `AxiDevice` as an example for inheritance. It abstracts entire AXI transactions. Specific devices can be derived from this class and define the behavior by overriding the `on_read` and `on_write` methods.

In [2]:
# Example for a class that abstracts axi4 lite
# devices. Could be part of a library
# and reused for multiple projects.
class AxiDevice:
    def __init__(self, axi: Axi4Light):
        # AxiDevice is built from two sequential contexts
        # proc_read and proc_write. They wait for axi requests,
        # call on_read/on_write and generate responses.
        # While this implementation used coroutines
        # both processes could also be written as
        # explicit state machines.

        @std.sequential(axi.clk, axi.reset)
        async def proc_read():
            req = await axi.await_read_request()
            data = self.on_read(req.addr)
            await axi.send_read_resp(data)

        @std.sequential(axi.clk, axi.reset)
        async def proc_write():
            req = await axi.await_write_request()
            self.on_write(req.addr, req.data)
            await axi.send_write_response()

    # derived classes should override
    # these methods to define the read/write action
    def on_read(self, addr: Signal[Unsigned]) -> BitVector[32]:
        return Null

    def on_write(self, addr: Signal[Unsigned], data: Signal[BitVector[32]]):
        pass

In [3]:
# AxiMemory inherits from AxiDevice
# it overrides the methods on_read and on_write
class AxiMemory(AxiDevice):
    def __init__(self, axi: Axi4Light, word_cnt: int):
        # initialize AxiDevice
        super().__init__(axi)

        # define memory as an array used by both
        # on_read and on_write

        self._memory = Signal[Array[BitVector[32], word_cnt]](name="memory")
        self._cnt = word_cnt

    def on_read(self, addr: Signal[Unsigned]) -> BitVector[32]:
        # Ignore the two lsbs because only word aligned
        # accesses are allowed. Interpret the bitvector as Unsigned.
        word_addr = addr.msb(rest=2).unsigned

        if word_addr < self._cnt:
            return self._memory[word_addr]
        else:
            return Null

    def on_write(self, addr: Signal[Unsigned], data: Signal[BitVector[32]]):
        word_addr = addr.msb(rest=2).unsigned

        if word_addr < self._cnt:
            self._memory[word_addr] <<= data

In [4]:
class MyEntity(Entity):
    clk = Port.input(Bit)
    reset = Port.input(Bit)

    def architecture(self):
        clk = std.Clock(self.clk)
        reset = std.Reset(self.reset)

        axi = Axi4Light.signal(clk, reset, 32)

        # Interconnect connects a single axi master
        # to multiple axi slaves. The master signals
        # are passed to the constructor. Slave ports
        # are dynamically created using the methods
        # reserve and/or connect.
        interconnect = Interconnect(axi)

        # reserve adds a new axi slave interface
        # with a given offset and address range to
        # the interconnect
        axi_1 = interconnect.reserve(0, 1024)

        # one instance of MyAxiMemory is connected
        # to the first reserved interface of interconnect
        AxiMemory(axi_1, 256)

        # reserve a second interface at on offset address of 1024
        axi_2 = interconnect.reserve(1024, 1024)

        # connect a second, smaller, instance of
        # MyAxiMemory to the second interface
        AxiMemory(axi_2, 64)

vhdl = std.VhdlCompiler.to_string(MyEntity)
print(vhdl)

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity MyEntity is
  port (
    clk : in std_logic;
    reset : in std_logic
    );
end MyEntity;


architecture arch_MyEntity of MyEntity is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal arready : std_logic := '0';
  signal rvalid : std_logic := '0';
  signal rdata : std_logic_vector(31 downto 0) := "00000000000000000000000000000000";
  signal rresp : std_logic_vector(1 downto 0) := "00";
  signal awready : std_logic := '0';
  signal wready : std_logic := '0';
  signal bvalid : std_logic := '0';
  signal bresp : std_logic_vector(1 downto 0) := "00";
  signal arready_1 : std_logic := '0';
  signal arvalid : std_logic := '0';
  signal araddr : std_logic_vector(9 downto 0) := "0000000000";
  signal arprot : std_logic_vector(2 downto 0) := "000";
  signal rready : st

---

In [5]:
class AxiRegister:
    def __init__(
        self,
        addr,
        signal: Signal[BitVector[32]] | None = None,
        read_only: bool = False,
        write_only: bool = False,
    ):
        assert not (read_only and write_only)
        self.addr = addr
        self.signal = Signal[BitVector[32]]() if signal is None else signal
        self.read_only = read_only
        self.write_only = write_only

    def on_read(self) -> BitVector[32]:
        return self.signal

    def on_write(self, data: Signal[BitVector[32]]):
        self.signal <<= data

# a AxiDevice made up of a list of AxiRegisters
class AxiRegisterDevice(AxiDevice):
    def __init__(self, axi: Axi4Light, registers: list[AxiRegister]):
        super().__init__(axi)
        self.registers = registers

    def on_read(self, addr: Signal[Unsigned]) -> BitVector[32]:
        result = Variable[BitVector[32]]()
        for reg in self.registers:
            if not reg.write_only:
                if addr.unsigned == reg.addr:
                    result @= reg.on_read()
                    break
        else:
            result @= Null
        return result

    def on_write(self, addr: Signal[Unsigned], data: Signal[BitVector[32]]):
        for reg in self.registers:
            if not reg.read_only:
                if addr.unsigned == reg.addr:
                    reg.on_write(data)
                    break

We can now use `AxiRegisterDevice` to implement memory mapped devices.

In [6]:
# AxiDevice with three registers
#   0x00: ctrl  - containing enable bit and prescaler
#   0x04: limit - count up to this limit then down to zero
#   0x08: count - current counter value

class UpDownCounter(AxiRegisterDevice):
    def __init__(self, axi: Axi4Light):
        # define three axi registers
        reg_ctrl = AxiRegister(0)
        reg_limit = AxiRegister(4)
        reg_count = AxiRegister(8, read_only=True)

        # extract fields from registers
        enable = reg_ctrl.signal[0]
        prescaler = reg_ctrl.signal[31:16].unsigned
        limit = reg_limit.signal.unsigned
        count = reg_count.signal.unsigned

        prescaler_tick = Signal[Bit](False)

        @std.sequential(axi.clk, axi.reset)
        def proc_prescaler(counter=Signal[Unsigned[16]](0)):
            if enable:
                if counter >= prescaler:
                    counter <<= 0
                    prescaler_tick.push = True
                else:
                    counter <<= counter + 1

        @std.sequential(axi.clk, axi.reset)
        def proc_counter(cnt_down=Signal[Bit](False)):
            nonlocal count

            if prescaler_tick:
                if cnt_down:
                    count <<= count - 1

                    if count - 1 == 0:
                        cnt_down <<= False
                else:
                    count <<= count + 1

                    if count + 1 >= limit:
                        cnt_down <<= True

        super().__init__(axi, registers=[reg_ctrl, reg_limit, reg_count])

In [7]:
class MyEntity(Entity):
    clk = Port.input(Bit)
    reset = Port.input(Bit)

    def architecture(self):
        clk = std.Clock(self.clk)
        reset = std.Reset(self.reset)

        # create the signals of an Axi4Light interface
        # with 16 Bit address width, in a real example
        # these would be connected to entity ports
        axi = Axi4Light.signal(clk, reset, addr_width=16)
        UpDownCounter(axi)

vhdl = std.VhdlCompiler.to_string(MyEntity)
print(vhdl)

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity MyEntity is
  port (
    clk : in std_logic;
    reset : in std_logic
    );
end MyEntity;


architecture arch_MyEntity of MyEntity is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal counter : unsigned(15 downto 0) := unsigned'("0000000000000000");
  signal prescaler_tick : std_logic := '0';
  signal enable : std_logic_vector(31 downto 0);
  signal cnt_down : std_logic := '0';
  signal count : std_logic_vector(31 downto 0);
  signal limit : std_logic_vector(31 downto 0);
  signal arready : std_logic := '0';
  type state_proc_read is (state_0, state_1, state_2);
  signal s_proc_read : state_proc_read := state_0;
  signal rdata : std_logic_vector(31 downto 0) := "00000000000000000000000000000000";
  signal rresp : std_logic_vector(1 downto 0) := "00";
  signal 